## Two Moons: Varying the number of samples

In [8]:
import sys  
sys.path.insert(0, '../../')

In [9]:
import optuna
import numpy as np
import pandas as pd

from sklearn.datasets import make_blobs, make_moons
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering, MeanShift, estimate_bandwidth
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_mutual_info_score as AMI
from rock import ROCK

import logging
import sys

In [10]:
seed = 0
n_samples = 1000
jitter = 15 * 0.01
datasets = []

n_samples_grid = [50, 100, 200, 400, 600, 800, 1000, 1200, 1400, 1600]

In [11]:
experiment = []
n_centers = 2
rock_results = []
other_results = []

In [12]:
for n in n_samples_grid:
    step = 0
    for s in range(seed, seed+10):
        #print(f'jitter: {j}, seed: {s}')
        dataset = make_moons(n_samples=n, noise=jitter, shuffle=True, random_state=s) 
        run = {}
        run['step'] = step
        run['n_samples'] = n
        run['n_centers'] = n_centers
        run['jitter'] = jitter
        scaler = StandardScaler()
        data = scaler.fit_transform(dataset[0])
        kmeans = KMeans(n_clusters=n_centers, random_state=seed).fit(data).labels_
        eps, min_pts = (0.2, 4)
        dbscan = DBSCAN(eps=eps, min_samples=min_pts).fit(data).labels_
        run['eps'] = eps
        run['min_pts'] = min_pts
        spectral = SpectralClustering(n_clusters=2, random_state=seed).fit(data).labels_
        bandwidth = estimate_bandwidth(data)

        run['bandwidth'] = bandwidth
        mean_shift = MeanShift(bandwidth=bandwidth).fit(data).labels_

        rock = ROCK(tmax=15).fit(data).labels_

        gt = dataset[1]
        rock_results.append(AMI(rock, gt))
        other_results.append(np.max([AMI(kmeans, gt), AMI(dbscan, gt), AMI(spectral, gt), AMI(mean_shift, gt)]))

        run['ROCK'] = AMI(rock, gt)
        run['K_MEANS'] = AMI(kmeans, gt)
        run['DBSCAN'] = AMI(dbscan, gt)
        run['SPECTRAL'] = AMI(spectral, gt)
        run['MEAN_SHIFT'] = AMI(mean_shift, gt)

        step += 1

        experiment.append(run)

In [13]:
pd.DataFrame(experiment).to_csv('../../results/analysis/two_moons_analysis_samples.csv')

In [14]:
pd.DataFrame(experiment)

,step,n_samples,n_centers,jitter,eps,min_pts,bandwidth,ROCK,K_MEANS,DBSCAN,SPECTRAL,MEAN_SHIFT
0,0,50,2,0.15,0.2,4,1.256490,0.470339,0.356112,0.098771,0.408445,0.526333
1,1,50,2,0.15,0.2,4,1.206272,0.234534,0.310651,0.000000,0.408445,0.280944
2,2,50,2,0.15,0.2,4,1.246717,0.724580,0.361061,0.098771,0.361061,0.361061
3,3,50,2,0.15,0.2,4,1.240091,0.337154,0.310651,0.000000,0.310651,0.310651
4,4,50,2,0.15,0.2,4,1.181165,0.356112,0.356112,0.000000,0.356112,0.356112
...,...,...,...,...,...,...,...,...,...,...,...,...
95,5,1600,2,0.15,0.2,4,1.246459,0.449125,0.369921,-0.000495,0.410721,0.418788
96,6,1600,2,0.15,0.2,4,1.244771,0.865812,0.396441,-0.000667,0.427418,0.437379
97,7,1600,2,0.15,0.2,4,1.252444,0.862723,0.390140,0.000509,0.432613,0.436253
98,8,1600,2,0.15,0.2,4,1.246762,0.480228,0.366998,-0.000816,0.415503,0.428755
